In [1]:
%config IPCompleter.greedy=True

from os import path
import torch
print(torch.__version__)
print(torch.cuda.is_available())

1.6.0
False


In [3]:
import numpy as np
import torchvision
from PIL import Image

from matplotlib import pyplot as plt;
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from torch.utils.data import DataLoader

In [4]:
from utils import *
from attacks import fgsm, bim, llc
from train_utils import train_model
from models.models import *

from nupic.torch.modules import (KWinners, SparseWeights, Flatten, SparseWeights2d, KWinners2d, 
                                 rezero_weights, update_boost_strength)

In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [7]:
transform = ToTensor()
ds = FashionMNIST('./data', train=True, target_transform=None, download=True, transform=transform)
ds_test = FashionMNIST('./data', train=False, target_transform=None, download=True, transform=transform)

In [8]:
test1000 = DataLoader(ds_test, 1000, shuffle=True)
for X, y in test1000:
    X, y = X.to(device), y.to(device)
    break

In [9]:
fcnet = FCNet()
sparse_fcnet = SparseFCNet()

fcnet.load_state_dict(torch.load("models/fcnet.pt", map_location=device))
sparse_fcnet.load_state_dict(torch.load("models/sparse_fcnet.pt", map_location=device))

<All keys matched successfully>

## FCNet vs SparseFCNet

### Clean images

In [10]:
from sklearn.metrics import accuracy_score

yp = fcnet(X).max(dim=1)[1].numpy()
syp = sparse_fcnet(X).max(dim=1)[1].numpy()

print("FCNet: {} ".format(accuracy_score(y.numpy(), yp)))
print("SparseFCNet: {} ".format(accuracy_score(y.numpy(), syp)))

FCNet: 0.881 
SparseFCNet: 0.866 


### Atak BIM

In [11]:
noise = bim(fcnet, nn.CrossEntropyLoss(), X, y, alpha=0.3, max_iter=4, clip_eps=8/255, do_clip=True)

yp = fcnet(X+noise).max(dim=1)[1].numpy()
syp = sparse_fcnet(X+noise).max(dim=1)[1].numpy()

print("FCNet: {} ".format(accuracy_score(y.numpy(), yp)))
print("SparseFCNet: {} ".format(accuracy_score(y.numpy(), syp)))

FCNet: 0.051 
SparseFCNet: 0.333 


In [12]:
noise = bim(sparse_fcnet, nn.CrossEntropyLoss(), X, y, alpha=0.3, max_iter=4, clip_eps=8/255, do_clip=True)

yp = fcnet(X+noise).max(dim=1)[1].numpy()
syp = sparse_fcnet(X+noise).max(dim=1)[1].numpy()

print("FCNet: {} ".format(accuracy_score(y.numpy(), yp)))
print("SparseFCNet: {} ".format(accuracy_score(y.numpy(), syp)))

FCNet: 0.44 
SparseFCNet: 0.08 


### Atak FGSM

In [30]:
noise = fgsm(fcnet, nn.CrossEntropyLoss(), X, y, alpha=0.3)

yp = fcnet(X+noise).max(dim=1)[1].numpy()
syp = sparse_fcnet(X+noise).max(dim=1)[1].numpy()

print("FCNet: {} ".format(accuracy_score(y.numpy(), yp)))
print("SparseFCNet: {} ".format(accuracy_score(y.numpy(), syp)))

FCNet: 0.0 
SparseFCNet: 0.002 


In [16]:
noise = fgsm(sparse_fcnet, nn.CrossEntropyLoss(), X, y, alpha=0.3)

yp = fcnet(X+noise).max(dim=1)[1].numpy()
syp = sparse_fcnet(X+noise).max(dim=1)[1].numpy()

print("FCNet: {} ".format(accuracy_score(y.numpy(), yp)))
print("SparseFCNet: {} ".format(accuracy_score(y.numpy(), syp)))

FCNet: 0.004 
SparseFCNet: 0.001 


### Atak LLC

In [17]:
noise = llc(fcnet, nn.CrossEntropyLoss(), X, max_iter=4, alpha=0.3, clip_eps=8/255, do_clip=True)

yp = fcnet(X+noise).max(dim=1)[1].numpy()
syp = sparse_fcnet(X+noise).max(dim=1)[1].numpy()

print("FCNet: {} ".format(accuracy_score(y.numpy(), yp)))
print("SparseFCNet: {} ".format(accuracy_score(y.numpy(), syp)))

FCNet: 0.529 
SparseFCNet: 0.705 


In [18]:
noise = llc(sparse_fcnet, nn.CrossEntropyLoss(), X, max_iter=4, alpha=0.3, clip_eps=8/255, do_clip=True)

yp = fcnet(X+noise).max(dim=1)[1].numpy()
syp = sparse_fcnet(X+noise).max(dim=1)[1].numpy()

print("FCNet: {} ".format(accuracy_score(y.numpy(), yp)))
print("SparseFCNet: {} ".format(accuracy_score(y.numpy(), syp)))

FCNet: 0.81 
SparseFCNet: 0.632 


## CNN vs SparseCNN

In [20]:
cnn = CNN()
sparse_cnn = SparseCNN()

cnn.load_state_dict(torch.load("models/cnn.pt", map_location=device))
sparse_cnn.load_state_dict(torch.load("models/sparse_cnn.pt", map_location=device))

<All keys matched successfully>

### Clean images

In [21]:
from sklearn.metrics import accuracy_score

yp = cnn(X).max(dim=1)[1].numpy()
syp = sparse_cnn(X).max(dim=1)[1].numpy()

print("CNN: {} ".format(accuracy_score(y.numpy(), yp)))
print("SparseCNN: {} ".format(accuracy_score(y.numpy(), syp)))

CNN: 0.905 
SparseCNN: 0.884 


### Atak BIM

In [22]:
noise = bim(cnn, nn.CrossEntropyLoss(), X, y, alpha=0.3, max_iter=4, clip_eps=8/255, do_clip=True)

yp = cnn(X+noise).max(dim=1)[1].numpy()
syp = sparse_cnn(X+noise).max(dim=1)[1].numpy()

print("CNN: {} ".format(accuracy_score(y.numpy(), yp)))
print("SparseCNN: {} ".format(accuracy_score(y.numpy(), syp)))

CNN: 0.492 
SparseCNN: 0.739 


In [23]:
noise = bim(sparse_cnn, nn.CrossEntropyLoss(), X, y, alpha=0.3, max_iter=4, clip_eps=8/255, do_clip=True)

yp = cnn(X+noise).max(dim=1)[1].numpy()
syp = sparse_cnn(X+noise).max(dim=1)[1].numpy()

print("CNN: {} ".format(accuracy_score(y.numpy(), yp)))
print("SparseCNN: {} ".format(accuracy_score(y.numpy(), syp)))

CNN: 0.775 
SparseCNN: 0.587 


### Atak FGSM

In [24]:
noise = fgsm(cnn, nn.CrossEntropyLoss(), X, y, alpha=0.3)

yp = cnn(X+noise).max(dim=1)[1].numpy()
syp = sparse_cnn(X+noise).max(dim=1)[1].numpy()

print("CNN: {} ".format(accuracy_score(y.numpy(), yp)))
print("SparseCNN: {} ".format(accuracy_score(y.numpy(), syp)))

CNN: 0.222 
SparseCNN: 0.463 


In [25]:
noise = fgsm(sparse_cnn, nn.CrossEntropyLoss(), X, y, alpha=0.3)

yp = cnn(X+noise).max(dim=1)[1].numpy()
syp = sparse_cnn(X+noise).max(dim=1)[1].numpy()

print("CNN: {} ".format(accuracy_score(y.numpy(), yp)))
print("SparseCNN: {} ".format(accuracy_score(y.numpy(), syp)))

CNN: 0.302 
SparseCNN: 0.368 


### Atak LLC

In [26]:
noise = llc(cnn, nn.CrossEntropyLoss(), X, max_iter=4, alpha=0.3, clip_eps=8/255, do_clip=True)

yp = cnn(X+noise).max(dim=1)[1].numpy()
syp = sparse_cnn(X+noise).max(dim=1)[1].numpy()

print("CNN: {} ".format(accuracy_score(y.numpy(), yp)))
print("SparseCNN: {} ".format(accuracy_score(y.numpy(), syp)))

CNN: 0.878 
SparseCNN: 0.88 


In [28]:
noise = llc(sparse_cnn, nn.CrossEntropyLoss(), X, max_iter=4, alpha=0.3, clip_eps=8/255, do_clip=True)

yp = cnn(X+noise).max(dim=1)[1].numpy()
syp = sparse_cnn(X+noise).max(dim=1)[1].numpy()

print("CNN: {} ".format(accuracy_score(y.numpy(), yp)))
print("SparseCNN: {} ".format(accuracy_score(y.numpy(), syp)))

CNN: 0.871 
SparseCNN: 0.872 
